In [1]:
config = {}
config['data'] = {"qm9_broad_ir_path":'/home2/kanakala.ganesh/ir_data/qm9_broad_ir.pkl',
                  "vocab_path":'/home2/kanakala.ganesh/CLIP_PART_1/data/qm9_vocab.pkl',
                  "datafiles" : {
                        'train': '/home2/kanakala.ganesh/ir_data/raw_train.pickle',
                        'test':  '/home2/kanakala.ganesh/ir_data/raw_test.pickle',
                        'val':   '/home2/kanakala.ganesh/ir_data/raw_val.pickle'
                        },
                  "normalization" : "unit",
                  "shuffle": True,
                  "batch_size":128,
                  "seq_len":70,
                  "splits":[0.9, 0.1, 0.1],
                  "num_workers":16
                }

In [2]:
from PrepareData import prepare_data
dataloaders, max_charge, num_species = prepare_data(config)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [3]:

for i , data in enumerate(dataloaders['train']):
    data
    for j in data:
        print(j, data[j].shape)
    break

index torch.Size([128])
decoder_inp torch.Size([128, 70])
decoder_tgt torch.Size([128, 70])
IR torch.Size([128, 1801])
tgt_padding_mask torch.Size([128, 70])
num_atoms torch.Size([128])
charges torch.Size([128, 29])
positions torch.Size([128, 29, 3])
one_hot torch.Size([128, 29, 5])
atom_mask torch.Size([128, 29])
edge_mask torch.Size([107648, 1])


In [4]:
config['molecule_encoder'] = {
    'attention': 1,
    'coords_weight' :1.0,
    'device': "cuda",
    'hidden_nf':256,
    'in_edge_nf':0,
    'in_node_nf':15,
    'n_layers': 3,
    'node_attr': 1,
    'output_size':512
}

In [5]:
import torch
from qm9.models import EGNN
from qm9 import utils as qm9_utils

device = torch.device("cuda")
dtype = torch.float32

In [6]:
model = EGNN(
             in_node_nf = config['molecule_encoder']['in_node_nf'], 
             in_edge_nf = config['molecule_encoder']['in_edge_nf'], 
             hidden_nf = config['molecule_encoder']['hidden_nf'], 
             device = torch.device(config['molecule_encoder']['device']), 
             n_layers = config['molecule_encoder']['n_layers'], 
             coords_weight = config['molecule_encoder']['coords_weight'],
             attention = config['molecule_encoder']['attention'], 
             node_attr = config['molecule_encoder']['node_attr'],
            output_size = config['molecule_encoder']['output_size'],
        )

In [9]:
def forward_pass(model, dataloader, max_charge, num_species):
    for i , data in enumerate(dataloader):
        data
        break
    
    batch_size = config['data']['batch_size']
    batch_size, n_nodes, _ = data['positions'].size()
    atom_positions = data['positions'].view(batch_size * n_nodes, -1).to(device, dtype)
    atom_mask = data['atom_mask'].view(batch_size * n_nodes, -1).to(device, dtype)
    edge_mask = data['edge_mask'].to(device, dtype)
    one_hot = data['one_hot'].to(device, dtype)
    charges = data['charges'].to(device, dtype)
    
    # print("batch_size", batch_size)
    # print("n_nodes", n_nodes)
    # print("atom_positions", atom_positions.shape)
    # print("atom_mask", atom_mask.shape)
    # print("edge_mask", edge_mask.shape)
    # print("one_hot", one_hot.shape)
    # print("charges", charges.shape)
    
    charge_scale = max_charge
    
    nodes = qm9_utils.preprocess_input(one_hot, 
                                   charges,
                                   2,
                                   charge_scale,
                                   device)

    nodes = nodes.view(batch_size * n_nodes, -1)
    # nodes = torch.cat([one_hot, charges], dim=1)
    edges = qm9_utils.get_adj_matrix(n_nodes, batch_size, device)
    
    preds = model(h0=nodes, 
             x=atom_positions, 
             edges=edges, 
             edge_attr=None, 
             node_mask=atom_mask, 
             edge_mask=edge_mask,
             n_nodes=n_nodes)
    
    print(preds.shape)
    return preds

In [11]:
out = forward_pass(model=model, 
             dataloader=dataloaders['train'],
             max_charge=max_charge,
             num_species=num_species)

torch.Size([128, 512])
